In [ ]:
# Python-MIP
import json
from mip import Model, xsum, BINARY, minimize, CONTINUOUS

# Load scenario
# https://github.com/power-grid-lib/pglib-uc
# Commit: cc3ccc1 (Verified)
# Directory: rts_gmlc
with open("./2020-01-27.json", "r") as f:
    scenario = json.load(f)

t_c = scenario["time_periods"]
d_c = scenario["demand"]
r_c = scenario["reserves"]
thermal_generators_list = [generator for generator in scenario["thermal_generators"].values()]
renewable_generators_list = [generator for generator in scenario["renewable_generators"].values()]

range_t = range(t_c)
range_g = range(len(thermal_generators_list))
range_w = range(len(renewable_generators_list))

# Startup categories index
range_s = []
range_s_1 = []  # Sg-1
# Piecewise interval index
range_l = []
# Cost at piecewise generation point
cp_c = []
# Power at piecewise generation point
pg_c = []
# Maximum/Minimum power output of thermal generator
pg_max_c = []
pg_min_c = []
# Must-run
u_must_c = []
# Limit of ramp-up/ramp-down
ru_l_c = []
rd_l_c = []
# Limit of start-up/shut-down
su_l_c = []
sd_l_c = []
# Minimum uptime/donwtime
ut_c = []
dt_c = []
# Time offline after which the startup category s becomes active
ts_c = []
# Startup cost in categories
cs_c = []

# Initial status
# Number of time periods the unit has been on/off prior to the initial time period
ut_0_c = []
dt_0_c = []
# Initial on/off status
u_0_c = []
# Initial Power output
pg_0_c = []
for g, generator in enumerate(thermal_generators_list):
    range_s.append(range(len(generator["startup"])))
    range_s_1.append(range(len(generator["startup"])-1))
    range_l.append(range(len(generator["piecewise_production"])))
    cp_c.append([piecewise_production["cost"] for piecewise_production in generator["piecewise_production"]])
    pg_c.append([piecewise_production["mw"] for piecewise_production in generator["piecewise_production"]])
    pg_min_c.append(generator["power_output_minimum"])
    pg_max_c.append(generator["power_output_maximum"])
    pg_0_c.append(generator["power_output_t0"])
    u_0_c.append(generator["unit_on_t0"])
    u_must_c.append(generator["must_run"])
    ru_l_c.append(generator["ramp_up_limit"])
    rd_l_c.append(generator["ramp_down_limit"])
    sd_l_c.append(generator["ramp_shutdown_limit"])
    su_l_c.append(generator["ramp_startup_limit"])
    ut_c.append(generator["time_up_minimum"])
    dt_c.append(generator["time_down_minimum"])
    ut_0_c.append(generator["time_up_t0"])
    dt_0_c.append(generator["time_down_t0"])
    ts_c.append([startup["lag"] for startup in generator["startup"]])
    cs_c.append([startup["cost"] for startup in generator["startup"]])

# Maximum/Minimum power output of renewable generator
pw_max_c = []
pw_min_c = []
for w, generator in enumerate(renewable_generators_list):
    pw_min_c.append(generator["power_output_minimum"])
    pw_max_c.append(generator["power_output_maximum"])

# Construct model
model = Model("unit-commitment-model")

# Add Variables
c_v = [[model.add_var(name=f"c[{g}]({t})", var_type=CONTINUOUS) for t in range_t] for g in range_g]
pg_v = [[model.add_var(name=f"pg[{g}]({t})", var_type=CONTINUOUS) for t in range_t] for g in range_g]
pw_v = [[model.add_var(name=f"pw[{w}]({t})", var_type=CONTINUOUS) for t in range_t] for w in range_w]
r_v = [[model.add_var(name=f"r[{g}]({t})", var_type=CONTINUOUS) for t in range_t] for g in range_g]
u_v = [[model.add_var(name=f"u[{g}]({t})", var_type=BINARY) for t in range_t] for g in range_g]
v_v = [[model.add_var(name=f"v[{g}]({t})", var_type=BINARY) for t in range_t] for g in range_g]
w_v = [[model.add_var(name=f"w[{g}]({t})", var_type=BINARY) for t in range_t] for g in range_g]
d_v = [[[model.add_var(name=f"d[{g}][{s}]({t})", var_type=BINARY) for t in range_t] for s in range_s[g]] for g in range_g]
l_v = [[[model.add_var(name=f"l[{g}][{l}]({t})", var_type=CONTINUOUS, ub=1.0) for t in range_t] for l in range_l[g]] for g in range_g]

# (1) Objective function
model.objective = minimize(xsum(c_v[g][t] + cp_c[g][0] * u_v[g][t] + xsum(cs_c[g][s] * d_v[g][s][t] for s in range_s[g]) for g in range_g for t in range_t))

# Add Constraints
for t in range_t:
    # (2) Demand
    model.add_constr(xsum(pg_v[g][t] + pg_min_c[g] * u_v[g][t] for g in range_g) + xsum(pw_v[w][t] for w in range_w) == d_c[t])
    # (3) Spinning reserve
    model.add_constr(xsum(r_v[g][t] for g in range_g) == r_c[t])

for g in range_g:
    # (4) Minimum up time constraint for generators which is on at initial state.
    if u_0_c[g] == 1:
        model.add_constr(xsum(u_v[g][t] - 1 for t in range(min(ut_c[g] - ut_0_c[g], t_c))) == 0)
    # (5) Minimum down time constraint for generators which is off at initial state.
    if u_0_c[g] == 0:
        model.add_constr(xsum(u_v[g][t] for t in range(min(dt_c[g] - dt_0_c[g], t_c))) == 0)
    # (6) Status change constraint at t=0.
    model.add_constr(u_v[g][0] - u_0_c[g] == v_v[g][0] - w_v[g][0])
    # (7) Startup category constraint based on initial state.
    model.add_constr(xsum(xsum(d_v[g][s][t] for t in range(max(0, ts_c[g][s + 1] - dt_0_c[g]), min(ts_c[g][s + 1] - 1, t_c))) for s in range_s_1[g]) == 0)
    # (8)(9) Ramp-up and Ramp-down limit from initial state.
    model.add_constr(pg_v[g][0] + r_v[g][0] - u_0_c[g] * (pg_0_c[g]- pg_min_c[g]) <= ru_l_c[g])
    model.add_constr(u_0_c[g] * (pg_0_c[g]- pg_min_c[g]) - pg_v[g][0] <= rd_l_c[g])
    # (10) Shutdown capability check (In order to shut down at the first time period, pg_0_c msut be less than equal sd_l_c.)
    model.add_constr(u_0_c[g] * (pg_0_c[g]- pg_min_c[g]) <= u_0_c[g] * (pg_max_c[g]- pg_min_c[g]) - max((pg_max_c[g] - sd_l_c[g]), 0) * w_v[g][0])


for g in range_g:
    for t in range(0, t_c - 1):
        # (18) The power limit at shutdown
        model.add_constr(pg_v[g][t] + r_v[g][t] <= (pg_max_c[g] - pg_min_c[g]) * u_v[g][t] - max(pg_max_c[g] - sd_l_c[g], 0) * w_v[g][t+1])

for g in range_g:
    for t in range(1, t_c):
        # (12) Status change (When u status change, start-up or shutdown)
        model.add_constr(u_v[g][t] - u_v[g][t-1] == v_v[g][t] - w_v[g][t])
        # (19)(20) Ramp-up and Ramp-down limit
        model.add_constr(pg_v[g][t] + r_v[g][t] - pg_v[g][t-1] <= ru_l_c[g])
        model.add_constr(pg_v[g][t-1] - pg_v[g][t] <= rd_l_c[g])

for g in range_g:
    for t in range(min(ut_c[g], t_c) - 1, t_c):
        # (13) Minimum up time constraint
        model.add_constr(xsum(v_v[g][i] for i in range(t - (min(ut_c[g], t_c) - 1), t + 1)) <= u_v[g][t])

for g in range_g:
    for t in range(min(dt_c[g], t_c) - 1, t_c):
        # (14) Minimum down time constraint
        model.add_constr(xsum(w_v[g][i] for i in range(t - (min(dt_c[g], t_c) - 1), t + 1)) <= 1 - u_v[g][t])

for g in range_g:
    for s in range_s_1[g]:
        for t in range(ts_c[g][s+1], t_c):
            # (15) Startup category constraint (When shutdown time is prior to the lag, the corresponding cost is required.)
            model.add_constr(d_v[g][s][t] <= xsum(w_v[g][t - i] for i in range(ts_c[g][s], ts_c[g][s+1])))

for g in range_g:
    for t in range_t:
        # (11) Must run constraint
        model.add_constr(u_v[g][t] >= u_must_c[g])
        # (16) Relation between start up status and start up category (Only one category is selectable.)
        model.add_constr(v_v[g][t] == xsum(d_v[g][s][t] for s in range_s[g]))
        # (17) The power limit at start-up
        model.add_constr(pg_v[g][t] + r_v[g][t] <= (pg_max_c[g] - pg_min_c[g]) * u_v[g][t] - max(pg_max_c[g] - su_l_c[g], 0) * v_v[g][t])
        # (21)(22)(23) Piecewise production cost interval formula
        model.add_constr(pg_v[g][t] == xsum((pg_c[g][l] - pg_c[g][0]) * l_v[g][l][t] for l in range_l[g]))
        model.add_constr(c_v[g][t] == xsum((cp_c[g][l] - cp_c[g][0]) * l_v[g][l][t] for l in range_l[g]))
        model.add_constr(u_v[g][t] == xsum(l_v[g][l][t] for l in range_l[g]))

for w in range_w:
    for t in range_t:
        # (24) Renewable energy upper limit and lower limit
        model.add_constr(pw_v[w][t] >= pw_min_c[w][t])
        model.add_constr(pw_v[w][t] <= pw_max_c[w][t])

In [ ]:
model.optimize(max_seconds=300)

In [ ]:
import io
buffer = io.StringIO()

for var in model.vars:
    buffer.write(f"{var}: {var.x}\n")
buffer.seek(0)
string = buffer.read()
with open("./test_result.txt", "w") as f:
    f.write(string)